In [96]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [97]:
names = ['S' + str(format(i, '02')) for i in range(5)] # + 1 because range starts at 0
numbers = [i for i in range(len(names))] # + 1 because range starts at 0
names.pop(0) # Pop the first element of the list to match the names of the files
names.pop(0)
print(names)
numbers.pop(0) # Pop the first element of the list to match the names of the files
numbers.pop(0)
print(numbers)

df_var = {}

# Create a list of the variables and its columns
variables = ['HR', 'EDA','TEMP']
final_columns = {
    'HR': ['HR',],
    'EDA': ['EDA'],
    'TEMP': ['TEMP'],
}
print(variables)

['S02', 'S03', 'S04']
[2, 3, 4]
['HR', 'EDA', 'TEMP']


In [98]:
# Import timelogs for visualization
# This DataFrame contains the time logs for each subject responses
timelogs = pd.read_csv('Time_logs.csv')
timelogs = timelogs.drop(0, axis=0) # Drop the first row
timelogs = timelogs.drop(['Session ID', 'Unnamed: 24'], axis=1)
timelogs

,S. ID.,Start Time,End Time,Consent,Unnamed: 5,Baseline/Questionniare,Unnamed: 7,Stroop Test,Unnamed: 9,Relax,...,Unnamed: 15,Hyperventilation,Unnamed: 17,Relax.2,Unnamed: 19,Questionniare,Unnamed: 21,Relax/Baseline,Unnamed: 23,Date
1,S01,9:22,10:22,9:22,9:27,9:39,9:43,9:29,9:34,9:34,...,9:59,10:00,10:02,10:02,10:07,10:08,10:12,10:12,10:22,7-Feb-2022
2,S02,9:48,10:51,9:48,9:50,9:52,10:02,10:03,10:08,10:08,...,10:29,10:29,10:31,10:31,10:36,10:36,10:41,10:41,10:51,7-Feb-2022
3,S03,10:50,11:50,10:50,10:53,10:55,11:02,11:04,11:09,11:09,...,11:28,11:28,11:30,11:30,11:35,11:35,11:40,11:40,11:50,7-Feb-2022
4,S04,11:25,12:28,11:25,11:30,11:30,11:40,11:40,11:45,11:45,...,12:05,12:05,12:07,12:07,12:12,12:12,12:18,12:18,12:28,7-Feb-2022
5,S05,9:05,10:06,9:05,9:10,9:12,9:22,9:23,9:29,9:29,...,9:49,9:49,9:51,9:51,9:56,9:56,10:01,10:01,10:06,14-Feb-2022
6,S06,9:41,10:40,9:41,9:43,9:44,9:54,9:55,10:01,10:02,...,10:23,10:23,10:25,10:25,10:30,10:30,10:35,10:35,10:40,14-Feb-2022
7,S07,10:25,11:26,10:25,10:29,10:31,10:41,10:43,10:48,10:48,...,11:08,11:09,11:11,11:11,11:16,11:16,11:21,11:21,11:26,14-Feb-2022
8,S08,12:08,1:05,12:08,12:10,12:12,12:22,12:23,12:28,12:28,...,12:48,12:48,12:50,12:50,12:55,12:55,1:00,1:00,1:05,14-Feb-2022
9,S09,12:34,1:31,12:34,12:36,12:38,12:48,12:49,12:54,12:54,...,1:14,1:14,1:16,1:16,1:21,1:21,1:26,1:26,1:31,14-Feb-2022
10,S10,1:11,2:05,1:11,1:13,1:14,1:23,1:23,1:28,1:29,...,1:48,1:49,1:51,1:51,1:56,1:56,1:59,1:59,2:05,14-Feb-2022


In [99]:
# Convert the time logs to unix timestamps
# This is done to be able to compare the time logs with the data
for column in timelogs.columns:
    unix_timestamps = []
    if column != 'Date' and column != 'S. ID.':
        timelogs[column] = timelogs.Date + ' ' + timelogs[column]
        times = timelogs[column].values.tolist()
        for date in times:
            dt_obj = datetime.strptime(date, '%d-%b-%Y %H:%M')
            unix_timestamps.append(int(dt_obj.timestamp()))
        timelogs[column] = unix_timestamps
timelogs = timelogs.drop(['Date', 'Start Time', 'End Time','Consent','Unnamed: 5'], axis=1)
timelogs

,S. ID.,Baseline/Questionniare,Unnamed: 7,Stroop Test,Unnamed: 9,Relax,Unnamed: 11,Interview,Unnamed: 13,Relax.1,Unnamed: 15,Hyperventilation,Unnamed: 17,Relax.2,Unnamed: 19,Questionniare,Unnamed: 21,Relax/Baseline,Unnamed: 23
1,S01,1644226740,1644226980,1644226140,1644226440,1644226440,1644226740,1644227040,1644227640,1644227640,1644227940,1644228000,1644228120,1644228120,1644228420,1644228480,1644228720,1644228720,1644229320
2,S02,1644227520,1644228120,1644228180,1644228480,1644228480,1644228780,1644228780,1644229440,1644229440,1644229740,1644229740,1644229860,1644229860,1644230160,1644230160,1644230460,1644230460,1644231060
3,S03,1644231300,1644231720,1644231840,1644232140,1644232140,1644232440,1644232440,1644232980,1644232980,1644233280,1644233280,1644233400,1644233400,1644233700,1644233700,1644234000,1644234000,1644234600
4,S04,1644233400,1644234000,1644234000,1644234300,1644234300,1644234600,1644234600,1644235200,1644235200,1644235500,1644235500,1644235620,1644235620,1644235920,1644235920,1644236280,1644236280,1644236880
5,S05,1644829920,1644830520,1644830580,1644830940,1644830940,1644831240,1644831240,1644831840,1644831840,1644832140,1644832140,1644832260,1644832260,1644832560,1644832560,1644832860,1644832860,1644833160
6,S06,1644831840,1644832440,1644832500,1644832860,1644832920,1644833220,1644833220,1644833880,1644833880,1644834180,1644834180,1644834300,1644834300,1644834600,1644834600,1644834900,1644834900,1644835200
7,S07,1644834660,1644835260,1644835380,1644835680,1644835680,1644835980,1644835980,1644836580,1644836580,1644836880,1644836940,1644837060,1644837060,1644837360,1644837360,1644837660,1644837660,1644837960
8,S08,1644840720,1644841320,1644841380,1644841680,1644841680,1644841980,1644842040,1644842580,1644842580,1644842880,1644842880,1644843000,1644843000,1644843300,1644843300,1644800400,1644800400,1644800700
9,S09,1644842280,1644842880,1644842940,1644843240,1644843240,1644843540,1644843540,1644800940,1644800940,1644801240,1644801240,1644801360,1644801360,1644801660,1644801660,1644801960,1644801960,1644802260
10,S10,1644801240,1644801780,1644801780,1644802080,1644802140,1644802440,1644802440,1644802980,1644802980,1644803280,1644803340,1644803460,1644803460,1644803760,1644803760,1644803940,1644803940,1644804300


In [100]:
def prepare_data(df_old, signal):
    df_new = df_old.copy()
    start_time = df_new.iloc[0,0]
    df_new.drop(0, axis=0, inplace=True)
    df_new.drop(1, axis=0, inplace=True)
    df_new.reset_index(drop=True, inplace=True)
    df_new.columns = [signal]
    if signal == 'HR':
        df_new['time'] = [int(start_time+(i)) for i in range(len(df_new))]
        df_new["label"] = ""
    else:
        df_new['time'] = np.repeat(np.arange(len(df_new)// 4 +1)+int(start_time), 4)[:len(df_new)]
        df_new["label"] = ""
    return df_new

In [101]:
def label(df,subject_number):
    flag = 0
    index = 1
    for column in range(timelogs.shape[1]):
        next_index = index + 1
        if index <= 17:
            val1 = timelogs.iloc[subject_number-1, index]
            val2 = timelogs.iloc[subject_number-1, next_index]
            if flag == 0:
                df.loc[(df['time'] >= val1) & (df['time'] <= val2), 'label'] = 0
                flag = 1
            elif flag == 1:
                df.loc[(df['time'] >= val1) & (df['time'] <= val2), 'label'] = 1
                flag = 0
        index += 2
    return df

In [115]:
ban = 0 
for signal in variables:
    for name in names:
        indx = names.index(name)
        df = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/{name}/{signal}.csv', header=None)
        df_new = prepare_data(df, signal)
        df_new = label(df_new, numbers[indx])
        df_new = df_new.replace('', np.nan, inplace=False)
        df_new = df_new.dropna(inplace=False)
        if ban == 0:
            df_var[signal] = df_new
            ban = 1
        else:
            df_var[signal] = pd.concat([df_var[signal], df_new], ignore_index=True)
    ban = 0

#df_var['HR'] = df_new
#df_var['HR'].to_csv('HR.csv', index=False)
#pd.DataFrame(df_var['HR'])#.to_csv('HR.csv', index=False)

In [103]:
## FIRST TRY
# df = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S03/HR.csv',header=None)
# df_HR = prepare_data(df, 'HR')
# df = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S02/HR.csv',header=None)
# df_HR2 = prepare_data(df, 'HR')
# df = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S01/HR.csv',header=None)
# df_HR3 = prepare_data(df, 'HR')

# df_HR = pd.concat([df_HR, df_HR2, df_HR3], ignore_index=True)
# df = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S03/EDA.csv',header=None)
# df_EDA = prepare_data(df, 'EDA')
# df = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S03/TEMP.csv',header=None)
# df_TEMP = prepare_data(df, 'TEMP')

# pd.set_option('display.max_rows', 100)
#df_HR = label(df_HR,3)
#df_HR = label(df_HR, numbers[2])
#df_EDA = label(df_EDA,3)
#df_TEMP = label(df_TEMP,3)
#df_HR

In [104]:
## FIRST TRY
# df_HR = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S03/HR.csv',header=None)
# start_time = df_HR.iloc[0,0]
# df_HR.drop(0, axis=0, inplace=True)
# df_HR.drop(1, axis=0, inplace=True)
# df_HR.reset_index(drop=True, inplace=True)
# df_HR.columns = ['HR']
# df_HR['time'] = [int(start_time+(i)) for i in range(len(df_HR))]
# df_HR["label"] = " "
# df_HR

In [105]:
## FIRST TRY
# df_EDA = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S03/EDA.csv',header=None)
# pd.set_option('display.max_rows', 100)
# start_time = df_EDA.iloc[0,0]
# df2 = df_EDA.T
# init = df2.pop(0)
# hz = df2.pop(1)
# df_EDA = df2.T
# time = np.repeat(np.arange(len(df_EDA)// 4 +1)+int(start_time), 4)[:len(df_EDA)]
# df_EDA['time'] = time
# df_EDA = df_EDA.rename(columns={0: 'EDA'}, inplace=False)
# df_EDA["label"] = " "
# df_EDA

## SECOND TRY
# df_EDA = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S03/EDA.csv',header=None)
# start_time = df_EDA.iloc[0,0]
# df_EDA.drop(0, axis=0, inplace=True)
# df_EDA.drop(1, axis=0, inplace=True)
# df_EDA.reset_index(drop=True, inplace=True)
# df_EDA.columns = ['EDA']
# df_EDA

In [106]:
## FIRST TRY
# df_TEMP = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S03/TEMP.csv',header=None)
# pd.set_option('display.max_rows', 100)
# start_time = df_TEMP.iloc[0,0]
# df2 = df_TEMP.T
# init = df2.pop(0)
# hz = df2.pop(1)
# df_TEMP = df2.T
# time = np.repeat(np.arange(len(df_TEMP)// 4 +1)+int(start_time), 4)[:len(df_TEMP)]
# df_TEMP['time'] = time
# df_TEMP = df_TEMP.rename(columns={0: 'TEMP'}, inplace=False)
# df_TEMP["label"] = " "
# df_TEMP

In [ ]:
# x_train = df_HR[df_HR['label'] != '']
# x_test = df_HR[df_HR['HR'] == '']
# y_train = df_HR[df_HR['label'] != '']
# y_test = df_HR[df_HR['label'] == '']

In [ ]:
#X = df_HR[['time', 'HR']]
#y = df_HR['label']
# test_sets = []
# # Split the data into groups based on the label column
# groups = [df_HR.loc[df_HR['label'] == label] for label in df_HR['label'].unique()]
# print(groups)
# # Loop through each group
# for group in groups:
    
#     # Determine the size of the group and the size of the test set
#     group_size = len(group)
#     test_size = int(group_size * 0.2)
    
#     # If the group size is less than or equal to 2, skip it
#     if group_size <= 2:
#         continue
# ------------------------------------------------------
# Define sliding window function
# def sliding_window(X, y, window_size):
#     X_window = []
#     y_window = []
#     for i in range(len(X) - window_size + 1):
#         X_window.append(X.iloc[i:i+window_size].values.flatten())
#         y_window.append(y.iloc[i+window_size-1])
#     return pd.DataFrame(X_window), pd.Series(y_window)

# # Generate input data using sliding window
# window_size = 10 # number of rows per window
# X_train_window, y_train_window = sliding_window(x_train, y_train, window_size)
# X_test_window, y_test_window = sliding_window(x_test, y_test, window_size)

# Implement screening ML algorithm (e.g. Decision Tree)
# from sklearn.tree import DecisionTreeClassifier

# # Fit the model
# dt = DecisionTreeClassifier(random_state=42)
# dt.fit(x_train, y_train)

# # Evaluate the model
# accuracy = dt.score(x_test, y_test)
# print('Accuracy:', accuracy)
# ------------------------------------------------------
from sklearn.metrics import classification_report

df_EDA['group'] = (df_EDA['label'] != df_EDA['label'].shift()).cumsum()

# Create a list of groups to exclude from the test set
groups = list(df_EDA['group'].unique())
print(groups)
exclude_groups = np.random.choice(groups, size=int(0.5*len(groups)), replace=False)
print(exclude_groups)
# Create a new column to indicate which rows are in the training set and which are in the test set
df_EDA['set'] = np.where(df_EDA['group'].isin(exclude_groups), 'test', 'train')

# Split the data into training and test sets
x_train_df = df_EDA[df_EDA['set'] == 'train'].drop(columns=['label','time','group','set'])
y_train_df = df_EDA[df_EDA['set'] == 'train'].drop(columns=['EDA','time','group','set'])
x_test_df = df_EDA[df_EDA['set'] == 'test'].drop(columns=['label','time','group','set'])
y_test_df = df_EDA[df_EDA['set'] == 'test'].drop(columns=['EDA','time','group','set'])
pd.set_option('display.max_rows', 100)
x_train_df

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[7 2 9 6]


,EDA
0,0.000000
1,0.137111
2,0.194774
3,0.196055
4,0.197337
...,...
10507,0.452604
10508,0.453885
10509,0.457730
10510,0.457730


In [ ]:
# Create a sliding window of size n for the training data
n = 2
x_train_data = []
y_train_data = []
for i in range(len(x_train_df)-n+1):
    x_window = x_train_df.iloc[i:i+n].values.flatten()
    y_window = y_train_df.iloc[i:i+n].values.flatten()
    x_train_data.append(x_window)
    y_train_data.append(y_window)
print(x_train_data)
print(y_train_data)

# Create a sliding window of size n for the test data
x_test_data = []
y_test_data = []
for i in range(len(x_test_df)-n+1):
    x_window = x_test_df.iloc[i:i+n].values.flatten()
    y_window = y_test_df.iloc[i:i+n].values.flatten()
    x_test_data.append(x_window)
    y_test_data.append(y_window)
print(x_test_data)
print(y_test_data)

[array([0.      , 0.137111]), array([0.137111, 0.194774]), array([0.194774, 0.196055]), array([0.196055, 0.197337]), array([0.197337, 0.198618]), array([0.198618, 0.197337]), array([0.197337, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.197337]), array([0.197337, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.197337]), array([0.197337, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.197337]), array([0.197337, 0.197337]), array([0.197337, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.197337]), array([0.197337, 0.198618]), array([0.198618, 0.198618]), array([0.198618, 0.197337]), array([0.197337, 0.197337]), array([0.197337, 0.197337]), array([0.197337, 0.198618]), array([0.1986

In [ ]:
# Split the training data into features and labels
# train_features = [x for x in x_train_data]
# train_labels = [y for y in y_train_data]
# print(train_features)
# print(train_labels)
# Train a random forest classifier
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=23)
clf.fit(x_train_df, y_train_df)

# Make predictions on the test data
# test_features = [x for x in x_test_data]
# test_labels = [y for y in y_test_data]
predictions = clf.predict(x_test_df)

# Print the classification report
print(classification_report(y_test_df, predictions))

C:\Users\Carlos\AppData\Local\Temp\ipykernel_21216\4201707418.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(x_train_df, y_train_df)


              precision    recall  f1-score   support

         0.0       0.60      0.59      0.60      3604
         1.0       0.15      0.16      0.16      1680

    accuracy                           0.45      5284
   macro avg       0.38      0.38      0.38      5284
weighted avg       0.46      0.45      0.46      5284

